In [ ]:
# https://sites.google.com/site/ryunosukehm/study/ml-with-python/impact-ss2017/hmm_toy_example
# https://www.kabuku.co.jp/developers/hmm
# http://keik-117.hatenablog.com/entry/2016/07/05/213903
%load_ext autoreload
%autoreload 2

import numpy as np
from hmmlearn import hmm
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from data_helper import *
from music_helper import *

In [ ]:
# https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn-hmm
# 隠れた状態は指番号 観測できるのは音符列
components = (5 * 5) * 2 # (1->1, 1->2, ..., 5->4, 5->5) * (上行, 下行)
model = hmm.GaussianHMM(n_components=components, covariance_type="diag")

model.startprob_ = np.ones(components) / components

trProbUpward = np.array([
    [0.00, 0.25, 0.25, 0.25, 0.25],
    [0.10, 0.00, 0.30, 0.30, 0.30],
    [0.10, 0.05, 0.00, 0.50, 0.35],
    [0.10, 0.05, 0.05, 0.00, 0.80],
    [0.20, 0.20, 0.20, 0.20, 0.20],
])
trProbDownward = np.array([
    [0.00, 0.50, 0.40, 0.10, 0.00],
    [0.90, 0.00, 0.05, 0.05, 0.00],
    [0.40, 0.55, 0.00, 0.05, 0.00],
    [0.25, 0.25, 0.50, 0.00, 0.00],
    [0.20, 0.15, 0.25, 0.40, 0.00],
])
t = np.array(
    [np.concatenate([np.zeros(i*5), trProbUpward[i] * 0.5, np.zeros((4-i)*5), np.zeros(i*5), trProbDownward[i] * 0.5, np.zeros((4-i)*5)]) for i in range(5)]
)
model.transmat_ = np.tile(t, (5 * 2, 1))
# ノードの出力
model.means_ = np.array([
    # 0 上行 1指 -> 
    [0],[1],[2],[3],[4],
    # 上行 2指 -> 
    [1],[0],[1],[2],[3],
    # 上行 3指 ->
    [1],[1],[0],[1],[2],
    # 上行 4指 -> 
    [1],[1],[1],[0],[1],
    # 上行 5指 -> 
    [5],[5],[5],[5],[5],
    # 25 下行 1指 -> 
    [0],[-1],[-1],[-1],[-1],
    
    [-1],[0],[-1],[-1],[-1],
    
    [-2],[-1],[0],[-1],[-1],
    
    [-3],[-2],[-1],[0],[-1],

    [-4],[-3],[-2],[-1],[0],
])
sigma = 1
covars = np.ones((components, 1)) * sigma
model.covars_ = covars

def to_inputs(notes):
    outputs = [0] * len(notes)
    for i in range(len(notes) - 1):
        outputs[i + 1] = note_pos(notes[i + 1]) - note_pos(notes[i])
    return outputs

In [ ]:
# 入力
stream = mu.converter.parse('./mxl/note_example1.xml')
notes = stream.flat.notes
inputs = to_inputs(notes)
# 予測
outputs = model.predict(np.array(inputs).reshape(len(inputs), 1))
fingers = outputs % 5 + 1
# 楽譜化
for i in range(len(notes)):
    notes[i].articulations.append(mu.articulations.Fingering(fingers[i]))
stream.show()

In [ ]:
# 入力
stream = mu.converter.parse('./tmp/Invention_no1.xml')
notes = stream.parts[0].flat.notes
inputs = to_inputs(notes)
# 予測
outputs = model.predict(np.array(inputs).reshape(len(inputs), 1))
fingers = outputs % 5 + 1
# 楽譜化
for i in range(len(notes)):
    notes[i].articulations.append(mu.articulations.Fingering(fingers[i]))
stream.parts[0].show()

# Automatic Determination of Piano Fingering based on Hidden Markov Model

In [ ]:
components = 5 * 5
# https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.GMMHMM
model = hmm.GMMHMM(n_components=components, n_mix=2, algorithm='viterbi', covariance_type='diag')
hmm._check_and_set_gaussian_n_features(model, np.zeros(shape = (0, 2)))

model.startprob_ = np.ones(components) / components

t = np.array([
    [0.20, 0.20, 0.20, 0.20, 0.20]+([0]*20),
    ([0]*5) +[0.20, 0.20, 0.20, 0.20, 0.20]+([0]*15),
    ([0]*10)+[0.21, 0.21, 0.21, 0.16, 0.21]+([0]*10),
    ([0]*15)+[0.23, 0.23, 0.16, 0.22, 0.16]+([0]*5),
    ([0]*20)+[0.21, 0.21, 0.21, 0.16, 0.21],
])
model.transmat_ = np.tile(t, (5, 1))

# ノードの出力　２次元混合ガウス分布
model.weights_ = np.array([
    # 1->1, 1->2, 1->3, 1->4, 1->5, 
    # 2->1, 2->2, 2->3, 2->4, 2->5, ...
#    [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00],  
    [1.00, 0.00], [0.81, 0.19], [0.89, 0.11], [0.91, 0.09], [0.95, 0.05], 
    [0.81, 0.19], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], 
    [0.89, 0.11], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], 
    [0.91, 0.09], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], 
    [0.95, 0.05], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], [1.00, 0.00], 
])
model.means_ = np.array([
    [[0, 0], [0, 0]],        [[42, 25], [-23, 25]], [[50, 30], [-16, 30]], [[85, 25], [-16, 25]], [[110, 0], [-21, 0]], 
    [[-42, -25], [23, -25]], [[0, 0], [0, 0]],      [[23, 10], [0, 0]],    [[50, 0], [0, 0]],     [[82, -25], [0, 0]],   
    [[-50, -30], [16, -30]], [[-23, 10], [0, 0]],   [[0, 0], [0, 0]],      [[18, -10], [0, 0]],   [[57, -25], [0, 0]], 
    [[-85, -25], [16, -25]], [[-50, 0], [0, 0]],    [[-18, 10], [0, 0]],   [[0, 0], [0, 0]],      [[20, -20], [0, 0]],   
    [[-110, -0], [21, -0]],  [[-82, 25], [0, 0]],   [[-57, 25], [0, 0]],   [[-20, 20], [0, 0]],   [[0, 0], [0, 0]], 
])
model.covars_ = np.array([
    [[5, 30], [0, 0]],      [[900, 30], [400, 30]], [[900, 30], [100, 30]], [[900, 30], [100, 30]], [[900, 30], [400, 30]], 
    [[900, 30], [400, 30]], [[5, 30], [0, 0]],      [[180, 30], [0, 0]],    [[200, 30], [0, 0]],    [[200, 30], [0, 0]], 
    [[900, 30], [100, 30]], [[180, 30], [0, 0]],    [[5, 30], [0, 0]],      [[190, 30], [0, 0]],    [[250, 30], [0, 0]], 
    [[900, 30], [100, 30]], [[200, 30], [0, 0]],    [[190, 30], [0, 0]],    [[5, 30], [0, 0]],      [[200, 30], [0, 0]], 
    [[900, 30], [400, 30]], [[200, 30], [0, 0]],    [[250, 30], [0, 0]],    [[200, 30], [0, 0]],    [[5, 30], [0, 0]], 
])

def to_inputs(notes):
    outputs = np.zeros(shape = (len(notes), 2))
    for i in range(len(notes) - 1):
        outputs[i + 1] = np.array(note_pos2(notes[i + 1])) - np.array(note_pos2(notes[i]))
    return outputs * 23 # mm単位

# Statistical Learning and Estimation of Piano Fingering

In [ ]:
# https://statpianofingering.github.io/demo.html
initProb = [0.39622, 0.169812, 0.18239, 0.0943429, 0.157234]

trProb = [
    [0.0337035, 0.397171, 0.198123, 0.150674, 0.220328],
    [0.317849, 0.0170221, 0.273942, 0.225422, 0.165765],
    [0.277559, 0.329458, 0.0244968, 0.131202, 0.237285],
    [0.384084, 0.16914, 0.242835, 0.0150975, 0.188843],
    [0.524839, 0.198501, 0.112848, 0.143469, 0.0203428],
]

outProb = [
    [3.92014e-06, 3.92014e-06, 0.0117643, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 0.00392406, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 0.00392406, 0.00392406, 3.92014e-06, 3.92014e-06, 0.00392406, 3.92014e-06, 3.92014e-06, 0.0156845, 3.92014e-06, 3.92014e-06, 0.0235248, 3.92014e-06, 3.92014e-06, 0.0509657, 0.0117643, 0.0078442, 0.0196046, 0.00392406, 3.92014e-06, 0.0117643, 0.00392406, 0.0117643, 0.0117643, 0.0352852, 0.0156845, 0.0784067, 0.0117643, 0.0274449, 0.313615, 0.0392053, 0.0274449, 0.0744866, 0.0235248, 0.0117643, 0.0274449, 0.0156845, 0.00392406, 0.0235248, 3.92014e-06, 3.92014e-06, 0.00392406, 0.0117643, 0.00392406, 3.92014e-06, 3.92014e-06, 3.92014e-06, 0.00392406, 3.92014e-06, 3.92014e-06, 0.0274449, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 0.00392406, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, 0.0156845, 3.92014e-06, 3.92014e-06, 3.92014e-06, 3.92014e-06, ],
    [3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 0.000333101, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 0.000333101, 0.00133141, 0.000998638, 0.000333101, 0.00066587, 0.00133141, 0.000333101, 3.32768e-07, 3.32768e-07, 0.00299525, 0.000998638, 3.32768e-07, 0.000998638, 0.0066557, 0.000998638, 0.016306, 0.030615, 0.00232971, 0.0196337, 0.0455896, 0.00798677, 0.133108, 0.0898478, 0.0695489, 0.189678, 0.0871857, 0.0715455, 0.120462, 0.0143094, 0.0106489, 0.0442585, 0.00133141, 0.00465909, 0.0106489, 0.00266248, 0.00299525, 0.000998638, 0.000333101, 3.32768e-07, 0.00266248, 0.000333101, 3.32768e-07, 3.32768e-07, 0.00066587, 0.000333101, 0.000333101, 3.32768e-07, 3.32768e-07, 0.000333101, 3.32768e-07, 3.32768e-07, 3.32768e-07, 0.000333101, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, 3.32768e-07, ],
    [6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 0.000667737, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 0.000667737, 0.000667737, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 0.00133481, 6.6707e-07, 6.6707e-07, 0.0113409, 0.0200128, 0.00133481, 0.0393578, 0.0346883, 0.00133481, 0.022681, 0.0140091, 0.00133481, 0.03202, 0.0526992, 0.0213469, 0.146089, 0.0847186, 0.0526992, 0.157429, 0.0266835, 0.0273505, 0.1094, 0.0286847, 0.0213469, 0.0426931, 0.0173445, 0.0106738, 0.00667137, 0.00333602, 0.00333602, 0.00266895, 6.6707e-07, 6.6707e-07, 6.6707e-07, 0.000667737, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 0.000667737, 6.6707e-07, 6.6707e-07, 0.000667737, 6.6707e-07, 6.6707e-07, 0.000667737, 6.6707e-07, 6.6707e-07, 0.000667737, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, 6.6707e-07, ],
    [8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 0.00175512, 0.00350936, 8.77121e-07, 0.00350936, 0.0114035, 8.77121e-07, 0.0166662, 0.00789497, 8.77121e-07, 0.00175512, 0.0245603, 0.00175512, 0.0289459, 0.00614073, 0.0105263, 0.0394713, 0.0385942, 0.00350936, 0.0842045, 0.0429798, 0.022806, 0.106133, 0.0824503, 0.0473654, 0.0850817, 0.0622765, 0.0421027, 0.0456112, 0.0192975, 0.00964921, 0.128938, 0.00175512, 0.00350936, 0.00175512, 0.00175512, 0.000877999, 0.00526361, 8.77121e-07, 8.77121e-07, 0.00175512, 0.00175512, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 0.00175512, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, 0.000877999, 8.77121e-07, 8.77121e-07, 8.77121e-07, 8.77121e-07, ],
    [5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 0.000600446, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 0.00120029, 5.99847e-07, 5.99847e-07, 5.99847e-07, 0.00120029, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 0.00180014, 0.000600446, 5.99847e-07, 0.000600446, 5.99847e-07, 0.00120029, 0.00239999, 0.00239999, 5.99847e-07, 0.0287932, 0.00359968, 0.00419953, 0.056986, 0.0551865, 0.0383908, 0.102574, 0.0305928, 0.0353915, 0.0563862, 0.0107978, 0.0089983, 0.469081, 0.0149968, 0.010198, 0.0131972, 0.00479937, 0.00539922, 0.00719876, 0.00299983, 0.00299983, 0.00839845, 0.000600446, 0.000600446, 0.00419953, 0.00239999, 0.00120029, 0.00180014, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 5.99847e-07, 0.00359968, 5.99847e-07, 0.000600446, 0.00120029, 0.000600446, ],
    [5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 0.00100145, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 0.000500978, 5.00477e-07, 0.000500978, 5.00477e-07, 0.00250289, 0.000500978, 5.00477e-07, 0.000500978, 0.00100145, 0.00150193, 0.00100145, 0.00300336, 0.00350384, 0.0240234, 0.00600623, 0.00700718, 0.0930893, 0.0575554, 0.0450434, 0.124119, 0.0535516, 0.100596, 0.21921, 0.021521, 0.108104, 0.022522, 0.00300336, 0.0480463, 0.0125124, 0.00100145, 0.0155153, 0.00500527, 0.00100145, 0.00800814, 0.00300336, 5.00477e-07, 0.000500978, 5.00477e-07, 5.00477e-07, 0.00150193, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 0.00250289, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 0.000500978, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, 5.00477e-07, ],
    [9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 0.00934702, 0.00934702, 9.33768e-06, 0.00934702, 0.0186847, 9.33768e-06, 0.00934702, 0.00934702, 0.0280224, 0.00934702, 0.0373601, 0.0186847, 0.606958, 0.00934702, 0.0186847, 0.0280224, 0.0186847, 0.00934702, 0.0373601, 0.00934702, 0.0186847, 0.00934702, 9.33768e-06, 9.33768e-06, 0.0186847, 0.00934702, 9.33768e-06, 9.33768e-06, 9.33768e-06, 0.00934702, 9.33768e-06, 9.33768e-06, 9.33768e-06, 0.0186847, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 0.0186847, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, 9.33768e-06, ],
    [5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 0.000581269, 0.00232334, 5.80689e-07, 0.000581269, 0.000581269, 0.00290402, 0.00116196, 0.000581269, 5.80689e-07, 0.0150985, 0.139947, 0.0836198, 0.238664, 0.111493, 0.129494, 0.1684, 0.0313578, 0.0377453, 0.0238088, 0.00116196, 0.00116196, 0.00696885, 0.00116196, 5.80689e-07, 5.80689e-07, 5.80689e-07, 0.000581269, 0.000581269, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, 5.80689e-07, ],
    [7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 0.000706376, 0.00141205, 0.000706376, 0.00282339, 0.00635174, 0.00282339, 0.00141205, 0.00776308, 0.0691564, 0.114319, 0.268155, 0.121376, 0.0797414, 0.229343, 0.0190538, 0.0254048, 0.0303445, 0.00282339, 0.00635174, 0.0049404, 0.00141205, 7.0567e-07, 7.0567e-07, 0.00141205, 7.0567e-07, 0.000706376, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 0.00141205, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, 7.0567e-07, ],
    [9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 0.000960567, 9.59607e-07, 9.59607e-07, 0.00287978, 9.59607e-07, 9.59607e-07, 0.00383939, 0.00192017, 0.0163143, 0.0143951, 0.0191931, 0.0374256, 0.226468, 0.02687, 0.0460621, 0.166013, 0.0422237, 0.118033, 0.129548, 0.036466, 0.063335, 0.0105566, 0.00383939, 0.00383939, 0.0191931, 9.59607e-07, 9.59607e-07, 0.000960567, 9.59607e-07, 9.59607e-07, 0.00192017, 0.000960567, 0.000960567, 0.004799, 9.59607e-07, 0.000960567, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, 9.59607e-07, ],
    [7.47891e-07, 0.000748639, 0.000748639, 7.47891e-07, 0.000748639, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 0.000748639, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 0.000748639, 7.47891e-07, 0.000748639, 0.000748639, 0.0127149, 0.00299231, 0.00149653, 0.011967, 0.0172022, 0.0149586, 0.039639, 0.0373953, 0.0373953, 0.121907, 0.0164544, 0.0426305, 0.185478, 0.0403869, 0.059832, 0.125646, 0.011967, 0.0605799, 0.0291685, 0.00149653, 0.0172022, 0.00972333, 0.00299231, 0.0321601, 0.0172022, 0.0037402, 0.0157065, 0.00747966, 0.00149653, 0.00149653, 0.011967, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 0.00149653, 7.47891e-07, 7.47891e-07, 7.47891e-07, 0.000748639, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, 7.47891e-07, ],
    [6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 0.000630713, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 0.000630713, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 0.000630713, 6.30083e-07, 6.30083e-07, 6.30083e-07, 0.000630713, 6.30083e-07, 0.000630713, 0.000630713, 0.0012608, 0.00378113, 0.010712, 0.00504129, 0.0151226, 0.0157527, 0.0170129, 0.0699398, 0.136099, 0.104594, 0.294879, 0.120346, 0.172643, 0.0189031, 0.000630713, 0.0012608, 0.00189088, 0.0012608, 0.000630713, 0.00189088, 6.30083e-07, 6.30083e-07, 0.000630713, 6.30083e-07, 0.000630713, 6.30083e-07, 0.000630713, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 0.000630713, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, 6.30083e-07, ],
    [8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 0.0169443, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 0.00847637, 8.4679e-06, 8.4679e-06, 8.4679e-06, 0.0508159, 0.0169443, 0.0169443, 8.4679e-06, 0.0169443, 8.4679e-06, 0.0254122, 0.0338801, 8.4679e-06, 0.0254122, 0.499615, 8.4679e-06, 0.0169443, 0.0508159, 0.0508159, 0.0169443, 8.4679e-06, 0.00847637, 8.4679e-06, 0.0592838, 0.00847637, 8.4679e-06, 0.042348, 8.4679e-06, 8.4679e-06, 0.00847637, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 0.0254122, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, 8.4679e-06, ],
    [1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 0.00316567, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 0.00632976, 0.00158363, 1.58205e-06, 1.58205e-06, 1.58205e-06, 0.0268964, 0.110745, 0.148714, 0.412916, 0.0791039, 0.115491, 0.0553732, 0.00791181, 0.00632976, 0.00949386, 0.00158363, 0.00158363, 0.00474772, 1.58205e-06, 1.58205e-06, 0.00316567, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 0.00316567, 0.00158363, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, 1.58205e-06, ],
    [8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 0.000875694, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 0.000875694, 8.74819e-07, 0.000875694, 8.74819e-07, 8.74819e-07, 0.00175051, 0.00437497, 0.00175051, 0.00787425, 0.00962389, 0.0708613, 0.102355, 0.275569, 0.111978, 0.139097, 0.173215, 0.0218714, 0.0122483, 0.0288699, 0.00524979, 0.00874907, 0.00962389, 0.00175051, 0.00350015, 0.00350015, 0.000875694, 8.74819e-07, 0.000875694, 8.74819e-07, 8.74819e-07, 0.000875694, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 0.000875694, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, 8.74819e-07, ],
    [0.00133303, 0.00199921, 6.66181e-07, 6.66181e-07, 0.00133303, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 0.000666847, 0.000666847, 0.00133303, 0.00133303, 6.66181e-07, 6.66181e-07, 0.00266539, 0.00133303, 0.00333157, 0.00133303, 0.00199921, 0.00866102, 0.0119919, 6.66181e-07, 0.0059963, 0.0626217, 0.00466393, 0.0173214, 0.0679512, 0.0253156, 0.0719482, 0.12391, 0.0659526, 0.0679512, 0.139233, 0.0313112, 0.0499643, 0.0959308, 0.00866102, 0.0173214, 0.0213185, 0.00466393, 0.00466393, 0.0219846, 0.00266539, 0.00533012, 0.00333157, 6.66181e-07, 0.0113257, 0.0059963, 6.66181e-07, 0.0093272, 0.00466393, 0.000666847, 0.00199921, 6.66181e-07, 6.66181e-07, 6.66181e-07, 0.00133303, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 0.000666847, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 0.00199921, 0.000666847, 6.66181e-07, 0.00133303, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, 6.66181e-07, ],
    [1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 0.00302681, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 0.00151416, 1.51265e-06, 1.51265e-06, 0.0060521, 1.51265e-06, 0.00151416, 1.51265e-06, 1.51265e-06, 1.51265e-06, 0.00151416, 1.51265e-06, 1.51265e-06, 1.51265e-06, 0.00756475, 0.00302681, 1.51265e-06, 0.00151416, 0.00151416, 0.00302681, 0.00302681, 0.00151416, 0.01059, 0.0499188, 0.0181533, 0.0211786, 0.213285, 0.110425, 0.0937856, 0.19362, 0.0907603, 0.0952982, 0.0226912, 0.00151416, 0.00907739, 0.00907739, 1.51265e-06, 1.51265e-06, 0.00453945, 0.00453945, 0.00151416, 0.00453945, 1.51265e-06, 1.51265e-06, 0.00151416, 1.51265e-06, 1.51265e-06, 0.00151416, 0.00302681, 1.51265e-06, 1.51265e-06, 1.51265e-06, 0.00151416, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 0.00302681, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, 1.51265e-06, ],
    [1.05364e-06, 0.00105469, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 0.00105469, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 0.00105469, 1.05364e-06, 0.00210833, 1.05364e-06, 1.05364e-06, 0.00632288, 1.05364e-06, 1.05364e-06, 0.0042156, 0.0105374, 0.00843015, 0.0168593, 0.113794, 0.0748093, 0.425671, 0.152778, 0.141188, 0.0200202, 0.0042156, 0.00105469, 1.05364e-06, 0.00948379, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 0.00105469, 1.05364e-06, 1.05364e-06, 0.00105469, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 0.00210833, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 0.00105469, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, 1.05364e-06, ],
    [1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 0.0169394, 0.0169394, 0.0169394, 0.0169394, 0.0169394, 0.0338619, 0.0677068, 0.0677068, 0.541536, 1.69225e-05, 0.0169394, 0.0338619, 0.0169394, 0.0169394, 1.69225e-05, 1.69225e-05, 1.69225e-05, 0.0338619, 0.0338619, 1.69225e-05, 0.0169394, 1.69225e-05, 1.69225e-05, 0.0169394, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 0.0169394, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, 1.69225e-05, ],
    [1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 0.0013562, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 0.00406588, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 0.0013562, 1.35484e-06, 0.00948525, 1.35484e-06, 0.00813041, 0.0216788, 0.0596144, 0.123292, 0.340067, 0.0826468, 0.101615, 0.165292, 0.0365821, 0.0121949, 0.0162595, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 0.0013562, 0.00271104, 1.35484e-06, 1.35484e-06, 0.00406588, 1.35484e-06, 1.35484e-06, 0.00271104, 1.35484e-06, 1.35484e-06, 0.0013562, 1.35484e-06, 1.35484e-06, 0.0013562, 1.35484e-06, 1.35484e-06, 0.0013562, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 0.0013562, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, 1.35484e-06, ],
    [0.00081637, 0.00326426, 0.000408389, 4.07981e-07, 0.00326426, 0.000408389, 4.07981e-07, 0.00122435, 0.00081637, 4.07981e-07, 0.00326426, 0.0024483, 0.00530416, 0.0106079, 0.00081637, 0.00081637, 0.0183596, 0.00693609, 0.00530416, 0.0216234, 0.0167276, 0.0110159, 0.0538539, 0.0232553, 0.0228474, 0.265596, 0.0244793, 0.0346788, 0.0844525, 0.0505901, 0.029783, 0.101996, 0.0354948, 0.0403906, 0.0558938, 0.00285628, 0.0044882, 0.0146877, 0.00367224, 0.00326426, 0.00612013, 0.00204031, 0.00122435, 0.0024483, 0.00081637, 0.00081637, 0.0106079, 0.000408389, 0.00204031, 0.00122435, 0.00122435, 0.00122435, 0.0024483, 0.000408389, 0.000408389, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 0.00081637, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, 4.07981e-07, ],
    [1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 0.00107972, 0.00215836, 0.00107972, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 0.00215836, 1.07864e-06, 0.003237, 1.07864e-06, 0.00431564, 0.00215836, 0.00107972, 0.00107972, 0.00539428, 0.00107972, 1.07864e-06, 0.003237, 1.07864e-06, 0.0086302, 0.00107972, 1.07864e-06, 0.0258884, 0.0657981, 0.0291244, 0.0819778, 0.107865, 0.0463826, 0.181213, 0.0550117, 0.0431467, 0.221122, 0.0474613, 0.0086302, 0.00970884, 0.0086302, 0.00431564, 0.00215836, 0.003237, 0.00431564, 0.00755156, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 0.00215836, 0.00215836, 1.07864e-06, 0.00215836, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 0.00107972, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 0.00107972, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, 1.07864e-06, ],
    [1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 0.0018991, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 0.00379629, 0.00948789, 1.8972e-06, 0.0151795, 0.00569349, 1.8972e-06, 0.0322543, 1.8972e-06, 0.0189739, 0.176441, 0.0910674, 0.100553, 0.252329, 0.119525, 0.077787, 0.0398431, 0.0246655, 0.0018991, 0.0132823, 0.0018991, 1.8972e-06, 0.00569349, 0.0018991, 0.0018991, 0.00379629, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, 1.8972e-06, ],
    [1.49233e-06, 0.00149382, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 0.00149382, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 0.00447848, 1.49233e-06, 0.00149382, 0.00149382, 0.00298615, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 0.00149382, 0.00447848, 1.49233e-06, 1.49233e-06, 0.0179095, 0.00746314, 0.0238788, 0.079095, 0.0761103, 0.070141, 0.404423, 0.171619, 0.0865566, 0.0164171, 0.00746314, 1.49233e-06, 0.00298615, 0.00298615, 0.00597081, 0.00298615, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 0.00149382, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 0.00149382, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 0.00149382, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, 1.49233e-06, ],
    [1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0210426, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0210426, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0105265, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0105265, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0210426, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0315586, 1.0516e-05, 1.0516e-05, 0.0105265, 0.0105265, 1.0516e-05, 0.0946547, 1.0516e-05, 0.0105265, 0.536328, 0.0105265, 0.0210426, 0.0525906, 0.0210426, 0.0315586, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0315586, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0105265, 1.0516e-05, 1.0516e-05, 0.0315586, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 0.0105265, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, 1.0516e-05, ],
]
df = pd.DataFrame(outProb, 
                  columns = list(itertools.product(range(-15, 16), range(-1, 2))), 
                  index = [f"{a}->{b}" for a, b in itertools.product(range(1, 6), range(1, 6))])

df = pd.DataFrame(trProb, 
                  columns = range(1,6), 
                  index = range(1,6))
sns.heatmap(df, vmax = 0.34, annot = True)

In [ ]:
# https://hmmlearn.readthedocs.io/en/latest/api.html#multinomialhmm
components = 5 * 5
model = hmm.MultinomialHMM(n_components=components, algorithm='viterbi')

model.startprob_ = np.array(initProb * 5) / 5

t = np.array(
    [np.concatenate([np.zeros(i*5), trp, np.zeros((4-i)*5)]) for i, trp in enumerate(trProb)]
)
model.transmat_ = np.tile(t, (5, 1))

widthX = 15
model.n_features = 3*(2*widthX+1)
model.emissionprob_ = outProb

def to_inputs(notes):
    outputs = np.zeros(shape = (len(notes), 2))
    for i in range(len(notes) - 1):
        outputs[i + 1] = np.array(note_pos2(notes[i + 1])) - np.array(note_pos2(notes[i]))
    return [int(3*(np.clip(o[0], -widthX, widthX) + widthX) + (o[1] + 1)) for o in outputs]

In [ ]:
# 入力
stream = mu.converter.parse('./mxl/note_example1.xml')
notes = stream.flat.notes
inputs = to_inputs(notes)

# 予測
outputs = model.predict(np.array(inputs).reshape(len(inputs), 1))
fingers = outputs % 5 + 1 
# 楽譜化
for i in range(len(notes)):
    notes[i].articulations.append(mu.articulations.Fingering(fingers[i]))
stream.show()

In [ ]:
# 入力
stream = mu.converter.parse('./tmp/Invention_no1.xml')
notes = stream.parts[0].flat.notes
inputs = to_inputs(notes)
# 予測
outputs = model.predict(np.array(inputs).reshape(len(inputs), 1))
fingers = outputs % 5 + 1
# 楽譜化
for i in range(len(notes)):
    notes[i].articulations.append(mu.articulations.Fingering(fingers[i]))
stream.parts[0].show()